In [1]:
# amount of columns and rows. Total amount of locations is gridsize^2
gridsize = 6

def generate_locations(size):
    rows = []
    cols = []
    
    for i in range(size):
        rows.append("r"+str(i+1))
        cols.append("c"+str(i+1))
        
    return rows, cols

rows, cols  = generate_locations(gridsize)
agent_actions = ["l", "r", "u", "d"]

### partition file: 

In [2]:
# print out partition file
print(".inputs: ", " ".join(rows + cols), sep='')
print(".outputs: ", " ".join(agent_actions), sep='')

.inputs: r1 r2 r3 r4 r5 r6 c1 c2 c3 c4 c5 c6
.outputs: l r u d


### mutual exclusion for actions/fluents + agent preconditions

In [3]:
def get_mutual_exclusion_actions(actions):
    all_actions_or = " | ".join(actions)
    action_combinations = []
    
    for i, action in enumerate(actions):
        for j in range(i+1, len(actions)):
            action_combinations.append("!(" + actions[i] + " & " + actions[j] + ")")
    
    action_comb_str = " & ".join(action_combinations)
    
    return all_actions_or, action_comb_str

def get_agent_preconditions():
    prec_list = []
    
    prec_list.append("(l -> !(" + cols[0] + "))")
    prec_list.append("(r -> !(" + cols[-1] + "))")
    prec_list.append("(u -> !(" + rows[0] + "))")
    prec_list.append("(d -> !(" + rows[-1] + "))")
    
    return prec_list  

### environment transitions (also states that remain the same)

In [4]:
def get_environment_transitions():
    trans_list = []
    
    # action left
    for i, col in enumerate(cols):
        if i == 0:
            pass
        elif i-1 == 0:
            trans_list.append("(" + col + " -> X(l -> " + cols[i-1] + "))")
        else:
            trans_list.append("(" + col + " -> X(l -> (" + cols[i-1] + " | " + cols[i-2] + ")))")
            
    # action right
    for i, col in enumerate(cols):
        if i == len(cols)-1:
            pass
        elif i == len(cols)-2:
            trans_list.append("(" + col + " -> X(r -> " + cols[i+1] + "))")
        else:
            trans_list.append("(" + col + " -> X(r -> (" + cols[i+1] + " | " + cols[i+2] + ")))")
    
    # action up
    for i, row in enumerate(rows):
        if i == 0:
            pass
        elif i-1 == 0:
            trans_list.append("(" + row + " -> X(u -> " + rows[i-1] + "))")
        else:
            trans_list.append("(" + row + " -> X(u -> (" + rows[i-1] + " | " + rows[i-2] + ")))")
            
    # action down
    for i, row in enumerate(rows):
        if i == len(rows)-1:
            pass
        elif i == len(rows)-2:
            trans_list.append("(" + row + " -> X(d -> " + rows[i+1] + "))")
        else:
            trans_list.append("(" + row + " -> X(d -> (" + rows[i+1] + " | " + rows[i+2] + ")))")
        
    return trans_list

def get_same_next_state():
    next_list = []
    
    # stay in same column when going up or down
    for col in cols:
        next_list.append("(" + col + " -> X(u -> " + col + "))")
        next_list.append("(" + col + " -> X(d -> " + col + "))")
    
    # stay in same row when going left or right
    for row in rows:
        next_list.append("(" + row + " -> X(l -> " + row + "))")
        next_list.append("(" + row + " -> X(r -> " + row + "))")
        
    return next_list

env_transitions = get_environment_transitions() + get_same_next_state()

# TODO: check if same next state can be removed
# env_transitions = get_environment_transitions()

### set init, goal, and game-over states below by hand (hardcoded):

In [5]:
def get_init():
    return ["r1", "c1"]

def get_goal():
    return ["r4", "c4"]

def get_game_over_states():
    return ["r3", "c2"]

In [6]:
# get mutual exclusion for agent actions and for environment rows + cols (can be only in 1 row and 1 column at a time)
mut_exc_agent_all, mut_exc_agent_comb = get_mutual_exclusion_actions(agent_actions)
mut_exc_env_rows_all, mutual_excl_env_rows_comb = get_mutual_exclusion_actions(rows)
mut_exc_env_cols_all, mutual_excl_env_cols_comb = get_mutual_exclusion_actions(cols)

### LTLf formula specification 
##### G(mutual exclusion agent actions) & ((initialization + G(mutual exclusion of where agent is allowed to be located by environment + environment transitions)) -> F(goal))

In [7]:
print("without death state: ")
print("G((", mut_exc_agent_all, ") & ", mut_exc_agent_comb, ") & ((", " & ".join(get_init()), " & G((", mut_exc_env_rows_all, ") & ", mutual_excl_env_rows_comb, ") & G((", mut_exc_env_cols_all, ") & ", mutual_excl_env_cols_comb, ") & G(", " & ".join(env_transitions), ")) -> F(", " & ".join(get_goal()),"))", sep='')
print()
print("with death state: ")
print("G((", mut_exc_agent_all, ") & ", mut_exc_agent_comb, ") & ((", " & ".join(get_init()), " & G((", mut_exc_env_rows_all, ") & ", mutual_excl_env_rows_comb, ") & G((", mut_exc_env_cols_all, ") & ", mutual_excl_env_cols_comb, ") & G(", " & ".join(env_transitions), ")) -> (F(", " & ".join(get_goal()),") & G(!(", " & ".join(get_game_over_states()),"))))", sep='')

without death state: 
G((l | r | u | d) & !(l & r) & !(l & u) & !(l & d) & !(r & u) & !(r & d) & !(u & d)) & ((r1 & c1 & G((r1 | r2 | r3 | r4 | r5 | r6) & !(r1 & r2) & !(r1 & r3) & !(r1 & r4) & !(r1 & r5) & !(r1 & r6) & !(r2 & r3) & !(r2 & r4) & !(r2 & r5) & !(r2 & r6) & !(r3 & r4) & !(r3 & r5) & !(r3 & r6) & !(r4 & r5) & !(r4 & r6) & !(r5 & r6)) & G((c1 | c2 | c3 | c4 | c5 | c6) & !(c1 & c2) & !(c1 & c3) & !(c1 & c4) & !(c1 & c5) & !(c1 & c6) & !(c2 & c3) & !(c2 & c4) & !(c2 & c5) & !(c2 & c6) & !(c3 & c4) & !(c3 & c5) & !(c3 & c6) & !(c4 & c5) & !(c4 & c6) & !(c5 & c6)) & G((c2 -> X(l -> c1)) & (c3 -> X(l -> (c2 | c1))) & (c4 -> X(l -> (c3 | c2))) & (c5 -> X(l -> (c4 | c3))) & (c6 -> X(l -> (c5 | c4))) & (c1 -> X(r -> (c2 | c3))) & (c2 -> X(r -> (c3 | c4))) & (c3 -> X(r -> (c4 | c5))) & (c4 -> X(r -> (c5 | c6))) & (c5 -> X(r -> c6)) & (r2 -> X(u -> r1)) & (r3 -> X(u -> (r2 | r1))) & (r4 -> X(u -> (r3 | r2))) & (r5 -> X(u -> (r4 | r3))) & (r6 -> X(u -> (r5 | r4))) & (r1 -> X(d -> (r2 

In [8]:
print("without death state (definitive formula, discussed with Fabio and Yves, 8-5-2024): ")
print("G((", mut_exc_agent_all, ") & ", mut_exc_agent_comb, ") & ((", " & ".join(get_init()), " & G((", mut_exc_env_rows_all, ") & ", mutual_excl_env_rows_comb, ") & G((", mut_exc_env_cols_all, ") & ", mutual_excl_env_cols_comb, ") & G(", " & ".join(env_transitions), ")) -> F(", " & ".join(get_goal()),"))", sep='')
print()

without death state (definitive formula, discussed with Fabio and Yves, 8-5-2024): 
G((l | r | u | d) & !(l & r) & !(l & u) & !(l & d) & !(r & u) & !(r & d) & !(u & d)) & ((r1 & c1 & G((r1 | r2 | r3 | r4 | r5 | r6) & !(r1 & r2) & !(r1 & r3) & !(r1 & r4) & !(r1 & r5) & !(r1 & r6) & !(r2 & r3) & !(r2 & r4) & !(r2 & r5) & !(r2 & r6) & !(r3 & r4) & !(r3 & r5) & !(r3 & r6) & !(r4 & r5) & !(r4 & r6) & !(r5 & r6)) & G((c1 | c2 | c3 | c4 | c5 | c6) & !(c1 & c2) & !(c1 & c3) & !(c1 & c4) & !(c1 & c5) & !(c1 & c6) & !(c2 & c3) & !(c2 & c4) & !(c2 & c5) & !(c2 & c6) & !(c3 & c4) & !(c3 & c5) & !(c3 & c6) & !(c4 & c5) & !(c4 & c6) & !(c5 & c6)) & G((c2 -> X(l -> c1)) & (c3 -> X(l -> (c2 | c1))) & (c4 -> X(l -> (c3 | c2))) & (c5 -> X(l -> (c4 | c3))) & (c6 -> X(l -> (c5 | c4))) & (c1 -> X(r -> (c2 | c3))) & (c2 -> X(r -> (c3 | c4))) & (c3 -> X(r -> (c4 | c5))) & (c4 -> X(r -> (c5 | c6))) & (c5 -> X(r -> c6)) & (r2 -> X(u -> r1)) & (r3 -> X(u -> (r2 | r1))) & (r4 -> X(u -> (r3 | r2))) & (r5 -> X(u -

## Below are outputs in separate parts (init, agent, environment, and goal)

In [23]:
# init
print("(", " & ".join(get_init()), ")", sep='')

(r1 & c1)


In [24]:
# agent
print("((", mut_exc_agent_all, ") & ", mut_exc_agent_comb, ")", sep='')

((l | r | u | d) & !(l & r) & !(l & u) & !(l & d) & !(r & u) & !(r & d) & !(u & d))


In [25]:
# environment
print("(((", mut_exc_env_rows_all, ") & ", mutual_excl_env_rows_comb, ") & ((", mut_exc_env_cols_all, ") & ", mutual_excl_env_cols_comb, ") & (", " & ".join(env_transitions), "))", sep='')

(((r1 | r2 | r3 | r4 | r5 | r6) & !(r1 & r2) & !(r1 & r3) & !(r1 & r4) & !(r1 & r5) & !(r1 & r6) & !(r2 & r3) & !(r2 & r4) & !(r2 & r5) & !(r2 & r6) & !(r3 & r4) & !(r3 & r5) & !(r3 & r6) & !(r4 & r5) & !(r4 & r6) & !(r5 & r6)) & ((c1 | c2 | c3 | c4 | c5 | c6) & !(c1 & c2) & !(c1 & c3) & !(c1 & c4) & !(c1 & c5) & !(c1 & c6) & !(c2 & c3) & !(c2 & c4) & !(c2 & c5) & !(c2 & c6) & !(c3 & c4) & !(c3 & c5) & !(c3 & c6) & !(c4 & c5) & !(c4 & c6) & !(c5 & c6)) & ((c2 -> X(l -> c1)) & (c3 -> X(l -> (c2 | c1))) & (c4 -> X(l -> (c3 | c2))) & (c5 -> X(l -> (c4 | c3))) & (c6 -> X(l -> (c5 | c4))) & (c1 -> X(r -> (c2 | c3))) & (c2 -> X(r -> (c3 | c4))) & (c3 -> X(r -> (c4 | c5))) & (c4 -> X(r -> (c5 | c6))) & (c5 -> X(r -> c6)) & (r2 -> X(u -> r1)) & (r3 -> X(u -> (r2 | r1))) & (r4 -> X(u -> (r3 | r2))) & (r5 -> X(u -> (r4 | r3))) & (r6 -> X(u -> (r5 | r4))) & (r1 -> X(d -> (r2 | r3))) & (r2 -> X(d -> (r3 | r4))) & (r3 -> X(d -> (r4 | r5))) & (r4 -> X(d -> (r5 | r6))) & (r5 -> X(d -> r6)) & (c1 -> X

In [26]:
# preconditions agent actions
print("(G((X(", cols[0], " | !", cols[0] , ")) -> ("," & ".join(get_agent_preconditions()),")))", sep="")

(G((X(c1 | !c1)) -> ((l -> !(c1)) & (r -> !(c6)) & (u -> !(r1)) & (d -> !(r6)))))


In [27]:
# goal
print("F(", " & ".join(get_goal()),")", sep='')

F(r4 & c4)
